# Ajuste de Hiperparâmetros (1.5)
Descrição:
Ajuste os parâmetros de uma MLP profunda com o Random Search para o conjunto de dados MNIST e veja se você consegue obter melhorar a precisão. Em seguida, aplicar a estratégia do funil, onde caso a arquitetura testada possua mais de 1 camada oculta, que a próxima seja menor em número de neurônios que a anterior. Depois compare as estratégias das camadas ocultas com o mesmo número de neurônios com o funil.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

# Aquisição de dados

In [2]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# pré-processamento

In [3]:
X_valid, X_train = X_train[:5000] / 255., X_train[5000:] / 255.
y_valid, y_train = y_train[:5000], y_train[5000:]
X_test = X_test / 255.

# Construindo e treinando o modelo

In [4]:
def build_model(n_hidden=1, n_neurons=300, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape= input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [5]:
def build_model2(n_hidden=1, n_neurons=300, learning_rate=3e-3, input_shape=[28, 28]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape= input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons / (layer + 1), activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [6]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_model)
model2 = keras.wrappers.scikit_learn.KerasClassifier(build_model2)

C:\Users\socia\AppData\Local\Temp\ipykernel_8780\3147291759.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = keras.wrappers.scikit_learn.KerasClassifier(build_model)
C:\Users\socia\AppData\Local\Temp\ipykernel_8780\3147291759.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model2 = keras.wrappers.scikit_learn.KerasClassifier(build_model2)


In [7]:
model.fit(X_train, y_train, epochs=30,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/30
1719/1719 [==============================] - 2s 1ms/step - loss: 1.0886 - accuracy: 0.7596 - val_loss: 0.5863 - val_accuracy: 0.8684
Epoch 2/30
1719/1719 [==============================] - 2s 993us/step - loss: 0.5058 - accuracy: 0.8749 - val_loss: 0.4217 - val_accuracy: 0.8894
Epoch 3/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4094 - accuracy: 0.8918 - val_loss: 0.3639 - val_accuracy: 0.9060
Epoch 4/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3656 - accuracy: 0.9004 - val_loss: 0.3314 - val_accuracy: 0.9110
Epoch 5/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3385 - accuracy: 0.9068 - val_loss: 0.3097 - val_accuracy: 0.9142
Epoch 6/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3186 - accuracy: 0.9111 - val_loss: 0.2932 - val_accuracy: 0.9190
Epoch 7/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3027 - accuracy: 0.9155 - val_loss: 0.2797 - val_accurac

In [8]:
model2.fit(X_train, y_train, epochs=30,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/30
1719/1719 [==============================] - 2s 1ms/step - loss: 1.0710 - accuracy: 0.7644 - val_loss: 0.5853 - val_accuracy: 0.8656
Epoch 2/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5075 - accuracy: 0.8717 - val_loss: 0.4253 - val_accuracy: 0.8916
Epoch 3/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4138 - accuracy: 0.8895 - val_loss: 0.3669 - val_accuracy: 0.9034
Epoch 4/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3707 - accuracy: 0.8984 - val_loss: 0.3353 - val_accuracy: 0.9104
Epoch 5/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3438 - accuracy: 0.9047 - val_loss: 0.3135 - val_accuracy: 0.9160
Epoch 6/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3242 - accuracy: 0.9092 - val_loss: 0.2980 - val_accuracy: 0.9192
Epoch 7/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3089 - accuracy: 0.9131 - val_loss: 0.2850 - val_accuracy:

# Otimizando os parâmetros com o RandomizedSearchCV

In [9]:
param_distribs = {
    "n_hidden": [1, 2, 3],
    "n_neurons": np.arange(195, 520).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(model, param_distribs, n_iter=10, scoring='accuracy', cv=3, verbose=2)

In [10]:
rnd_search_cv.fit(X_train, y_train, epochs=30,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10),
                             keras.callbacks.ModelCheckpoint("melhor_modelo_mlp_hiperparameters2.h5", save_best_only=True)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/30
1146/1146 [==============================] - 3s 2ms/step - loss: 1.9496 - accuracy: 0.5161 - val_loss: 1.4741 - val_accuracy: 0.7384
Epoch 2/30
1146/1146 [==============================] - 2s 2ms/step - loss: 1.0649 - accuracy: 0.7901 - val_loss: 0.7592 - val_accuracy: 0.8288
Epoch 3/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.6429 - accuracy: 0.8443 - val_loss: 0.5391 - val_accuracy: 0.8648
Epoch 4/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4992 - accuracy: 0.8684 - val_loss: 0.4445 - val_accuracy: 0.8848
Epoch 5/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4294 - accuracy: 0.8812 - val_loss: 0.3919 - val_accuracy: 0.8958
Epoch 6/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3874 - accuracy: 0.8917 - val_loss: 0.3571 - val_accuracy: 0.9004
Epoch 7/30
1146/1146 [==============================] - 2s 2ms/step - loss:

1146/1146 [==============================] - 2s 2ms/step - loss: 0.2056 - accuracy: 0.9412 - val_loss: 0.2052 - val_accuracy: 0.9412
Epoch 27/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.2018 - accuracy: 0.9431 - val_loss: 0.2022 - val_accuracy: 0.9414
Epoch 28/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.1977 - accuracy: 0.9439 - val_loss: 0.1980 - val_accuracy: 0.9426
Epoch 29/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.1939 - accuracy: 0.9455 - val_loss: 0.1949 - val_accuracy: 0.9444
Epoch 30/30
573/573 [==============================] - 1s 851us/step
[CV] END learning_rate=0.00127630734758697, n_hidden=3, n_neurons=359; total time= 1.2min
Epoch 1/30
1146/1146 [==============================] - 3s 3ms/step - loss: 1.9964 - accuracy: 0.5039 - val_loss: 1.5670 - val_accuracy: 0.7034
Epoch 2/30
1146/1146 [==============================] - 2s 2ms/step - loss: 1.1296 - accuracy: 0.7717 - val_loss: 0.7886 - val_ac

Epoch 21/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3093 - accuracy: 0.9130 - val_loss: 0.2961 - val_accuracy: 0.9174
Epoch 22/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3048 - accuracy: 0.9149 - val_loss: 0.2916 - val_accuracy: 0.9198
Epoch 23/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3006 - accuracy: 0.9161 - val_loss: 0.2879 - val_accuracy: 0.9214
Epoch 24/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2965 - accuracy: 0.9170 - val_loss: 0.2842 - val_accuracy: 0.9216
Epoch 25/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2926 - accuracy: 0.9181 - val_loss: 0.2808 - val_accuracy: 0.9224
Epoch 26/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2889 - accuracy: 0.9191 - val_loss: 0.2775 - val_accuracy: 0.9238
Epoch 27/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2853 - accuracy: 0.9203 - val_loss: 0.2743 - val_ac

1146/1146 [==============================] - 1s 1ms/step - loss: 0.3476 - accuracy: 0.9045 - val_loss: 0.3234 - val_accuracy: 0.9136
Epoch 16/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3408 - accuracy: 0.9060 - val_loss: 0.3173 - val_accuracy: 0.9154
Epoch 17/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3347 - accuracy: 0.9066 - val_loss: 0.3119 - val_accuracy: 0.9162
Epoch 18/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3291 - accuracy: 0.9088 - val_loss: 0.3069 - val_accuracy: 0.9168
Epoch 19/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3239 - accuracy: 0.9101 - val_loss: 0.3021 - val_accuracy: 0.9178
Epoch 20/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3190 - accuracy: 0.9110 - val_loss: 0.2972 - val_accuracy: 0.9206
Epoch 21/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.3145 - accuracy: 0.9119 - val_loss: 0.2937 - val_accuracy: 0.92

1146/1146 [==============================] - 2s 2ms/step - loss: 0.0608 - accuracy: 0.9831 - val_loss: 0.0919 - val_accuracy: 0.9728
Epoch 10/30
1146/1146 [==============================] - 3s 2ms/step - loss: 0.0531 - accuracy: 0.9856 - val_loss: 0.0878 - val_accuracy: 0.9754
Epoch 11/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.0454 - accuracy: 0.9879 - val_loss: 0.0865 - val_accuracy: 0.9748
Epoch 12/30
1146/1146 [==============================] - 3s 2ms/step - loss: 0.0396 - accuracy: 0.9895 - val_loss: 0.0784 - val_accuracy: 0.9756
Epoch 13/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.0343 - accuracy: 0.9917 - val_loss: 0.0814 - val_accuracy: 0.9768
Epoch 14/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.0297 - accuracy: 0.9931 - val_loss: 0.0828 - val_accuracy: 0.9760
Epoch 15/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.0260 - accuracy: 0.9944 - val_loss: 0.0788 - val_accuracy: 0.97

...

Epoch 17/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.1808 - accuracy: 0.9487 - val_loss: 0.1828 - val_accuracy: 0.9486
Epoch 18/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.1746 - accuracy: 0.9499 - val_loss: 0.1768 - val_accuracy: 0.9492
Epoch 19/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.1687 - accuracy: 0.9515 - val_loss: 0.1728 - val_accuracy: 0.9528
Epoch 20/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.1626 - accuracy: 0.9536 - val_loss: 0.1677 - val_accuracy: 0.9514
Epoch 21/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.1573 - accuracy: 0.9555 - val_loss: 0.1620 - val_accuracy: 0.9542
Epoch 22/30
1146/1146 [==============================] - 4s 3ms/step - loss: 0.1519 - accuracy: 0.9569 - val_loss: 0.1608 - val_accuracy: 0.9572
Epoch 23/30
1146/1146 [==============================] - 3s 3ms/step - loss: 0.1471 - accuracy: 0.9585 - val_loss: 0.1545 - val_ac

1146/1146 [==============================] - 1s 1ms/step - loss: 0.1476 - accuracy: 0.9579 - val_loss: 0.1517 - val_accuracy: 0.9590
Epoch 12/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1389 - accuracy: 0.9609 - val_loss: 0.1456 - val_accuracy: 0.9588
Epoch 13/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1303 - accuracy: 0.9641 - val_loss: 0.1392 - val_accuracy: 0.9612
Epoch 14/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1232 - accuracy: 0.9664 - val_loss: 0.1337 - val_accuracy: 0.9608
Epoch 15/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1163 - accuracy: 0.9678 - val_loss: 0.1303 - val_accuracy: 0.9638
Epoch 16/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.1102 - accuracy: 0.9699 - val_loss: 0.1241 - val_accuracy: 0.9660
Epoch 17/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1044 - accuracy: 0.9717 - val_loss: 0.1206 - val_accuracy: 0.96

1146/1146 [==============================] - 1s 1ms/step - loss: 0.2446 - accuracy: 0.9300 - val_loss: 0.2221 - val_accuracy: 0.9382
Epoch 6/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2237 - accuracy: 0.9359 - val_loss: 0.2071 - val_accuracy: 0.9446
Epoch 7/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2051 - accuracy: 0.9414 - val_loss: 0.1905 - val_accuracy: 0.9460
Epoch 8/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1891 - accuracy: 0.9460 - val_loss: 0.1783 - val_accuracy: 0.9508
Epoch 9/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1756 - accuracy: 0.9495 - val_loss: 0.1686 - val_accuracy: 0.9528
Epoch 10/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.1625 - accuracy: 0.9534 - val_loss: 0.1596 - val_accuracy: 0.9572
Epoch 11/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1521 - accuracy: 0.9561 - val_loss: 0.1529 - val_accuracy: 0.9566
E

1719/1719 [==============================] - 3s 2ms/step - loss: 0.0035 - accuracy: 0.9999 - val_loss: 0.0668 - val_accuracy: 0.9830


RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002290FE71C70>,
                   param_distributions={'learning_rate': [0.00043610409598925754,
                                                          0.0008839634716156907,
                                                          0.008852171010637948,
                                                          0.0042164798490471325,
                                                          0.003672217490113264,
                                                          0.0033486924900513896,
                                                          0.008702883790264666,
                                                          0.010737910753092461,
                                                          0.00045142191270034247,
                                                          0.02047926...
                                                          0.00035530297

In [11]:
param_distribs = {
    "n_hidden": [1, 2, 3],
    "n_neurons": np.arange(200, 500).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(model2, param_distribs, n_iter=10, scoring='accuracy', cv=3, verbose=2)

In [12]:
rnd_search_cv.fit(X_train, y_train, epochs=30,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10),
                             keras.callbacks.ModelCheckpoint("melhor_modelo_mlp_hiperparameters2_funil.h5", save_best_only=True)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/30
1146/1146 [==============================] - 2s 2ms/step - loss: 1.9677 - accuracy: 0.4747 - val_loss: 1.6420 - val_accuracy: 0.6806
Epoch 2/30
1146/1146 [==============================] - 2s 1ms/step - loss: 1.4169 - accuracy: 0.7417 - val_loss: 1.2116 - val_accuracy: 0.7746
Epoch 3/30
1146/1146 [==============================] - 2s 2ms/step - loss: 1.0828 - accuracy: 0.7968 - val_loss: 0.9569 - val_accuracy: 0.8174
Epoch 4/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.8844 - accuracy: 0.8249 - val_loss: 0.8038 - val_accuracy: 0.8362
Epoch 5/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.7611 - accuracy: 0.8415 - val_loss: 0.7048 - val_accuracy: 0.8514
Epoch 6/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.6787 - accuracy: 0.8524 - val_loss: 0.6361 - val_accuracy: 0.8618
Epoch 7/30
1146/1146 [==============================] - 2s 1ms/step - loss:

1146/1146 [==============================] - 2s 2ms/step - loss: 0.3590 - accuracy: 0.9041 - val_loss: 0.3426 - val_accuracy: 0.9102
Epoch 27/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3546 - accuracy: 0.9048 - val_loss: 0.3385 - val_accuracy: 0.9102
Epoch 28/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3505 - accuracy: 0.9062 - val_loss: 0.3345 - val_accuracy: 0.9108
Epoch 29/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3465 - accuracy: 0.9073 - val_loss: 0.3308 - val_accuracy: 0.9122
Epoch 30/30
573/573 [==============================] - 0s 654us/step
[CV] END learning_rate=0.000616494434253204, n_hidden=1, n_neurons=493; total time=  54.5s
Epoch 1/30
1146/1146 [==============================] - 2s 2ms/step - loss: 2.0157 - accuracy: 0.4011 - val_loss: 1.7059 - val_accuracy: 0.6588
Epoch 2/30
1146/1146 [==============================] - 2s 2ms/step - loss: 1.4867 - accuracy: 0.7164 - val_loss: 1.2747 - val_a

1146/1146 [==============================] - 1s 1ms/step - loss: 0.2242 - accuracy: 0.9355 - val_loss: 0.2173 - val_accuracy: 0.9380
Epoch 21/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2185 - accuracy: 0.9378 - val_loss: 0.2111 - val_accuracy: 0.9374
Epoch 22/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2131 - accuracy: 0.9390 - val_loss: 0.2064 - val_accuracy: 0.9414
Epoch 23/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2083 - accuracy: 0.9405 - val_loss: 0.2017 - val_accuracy: 0.9404
Epoch 24/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2031 - accuracy: 0.9423 - val_loss: 0.1982 - val_accuracy: 0.9420
Epoch 25/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1986 - accuracy: 0.9431 - val_loss: 0.1943 - val_accuracy: 0.9442
Epoch 26/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1940 - accuracy: 0.9449 - val_loss: 0.1918 - val_accuracy: 0.94

1146/1146 [==============================] - 1s 1ms/step - loss: 0.2650 - accuracy: 0.9247 - val_loss: 0.2516 - val_accuracy: 0.9300
Epoch 15/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2569 - accuracy: 0.9271 - val_loss: 0.2434 - val_accuracy: 0.9344
Epoch 16/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2490 - accuracy: 0.9295 - val_loss: 0.2370 - val_accuracy: 0.9360
Epoch 17/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2419 - accuracy: 0.9317 - val_loss: 0.2297 - val_accuracy: 0.9378
Epoch 18/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2352 - accuracy: 0.9332 - val_loss: 0.2252 - val_accuracy: 0.9376
Epoch 19/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.2288 - accuracy: 0.9356 - val_loss: 0.2188 - val_accuracy: 0.9404
Epoch 20/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.2230 - accuracy: 0.9361 - val_loss: 0.2153 - val_accuracy: 0.94

1146/1146 [==============================] - 2s 2ms/step - loss: 0.4727 - accuracy: 0.8779 - val_loss: 0.4427 - val_accuracy: 0.8882
Epoch 9/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4443 - accuracy: 0.8833 - val_loss: 0.4167 - val_accuracy: 0.8922
Epoch 10/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4218 - accuracy: 0.8876 - val_loss: 0.3972 - val_accuracy: 0.8968
Epoch 11/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.4036 - accuracy: 0.8912 - val_loss: 0.3798 - val_accuracy: 0.8992
Epoch 12/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3884 - accuracy: 0.8946 - val_loss: 0.3660 - val_accuracy: 0.9040
Epoch 13/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3756 - accuracy: 0.8972 - val_loss: 0.3543 - val_accuracy: 0.9058
Epoch 14/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.3643 - accuracy: 0.8995 - val_loss: 0.3440 - val_accuracy: 0.907

...

1146/1146 [==============================] - 1s 1ms/step - loss: 0.1907 - accuracy: 0.9472 - val_loss: 0.1857 - val_accuracy: 0.9496
Epoch 11/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1814 - accuracy: 0.9491 - val_loss: 0.1788 - val_accuracy: 0.9502
Epoch 12/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1723 - accuracy: 0.9524 - val_loss: 0.1725 - val_accuracy: 0.9530
Epoch 13/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1645 - accuracy: 0.9541 - val_loss: 0.1671 - val_accuracy: 0.9542
Epoch 14/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1571 - accuracy: 0.9567 - val_loss: 0.1619 - val_accuracy: 0.9554
Epoch 15/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1503 - accuracy: 0.9582 - val_loss: 0.1577 - val_accuracy: 0.9570
Epoch 16/30
1146/1146 [==============================] - 1s 1ms/step - loss: 0.1442 - accuracy: 0.9604 - val_loss: 0.1501 - val_accuracy: 0.96

1146/1146 [==============================] - 2s 1ms/step - loss: 1.1435 - accuracy: 0.7917 - val_loss: 1.0197 - val_accuracy: 0.8094
Epoch 5/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.9422 - accuracy: 0.8193 - val_loss: 0.8554 - val_accuracy: 0.8318
Epoch 6/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.8055 - accuracy: 0.8367 - val_loss: 0.7432 - val_accuracy: 0.8446
Epoch 7/30
1146/1146 [==============================] - 2s 2ms/step - loss: 0.7109 - accuracy: 0.8475 - val_loss: 0.6640 - val_accuracy: 0.8540
Epoch 8/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.6428 - accuracy: 0.8555 - val_loss: 0.6052 - val_accuracy: 0.8614
Epoch 9/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.5920 - accuracy: 0.8632 - val_loss: 0.5606 - val_accuracy: 0.8658
Epoch 10/30
1146/1146 [==============================] - 2s 1ms/step - loss: 0.5527 - accuracy: 0.8675 - val_loss: 0.5254 - val_accuracy: 0.8706
Ep

1146/1146 [==============================] - 2s 1ms/step - loss: 0.3446 - accuracy: 0.9060 - val_loss: 0.3313 - val_accuracy: 0.9082
Epoch 30/30
573/573 [==============================] - 0s 679us/step
[CV] END learning_rate=0.00039732043181284926, n_hidden=2, n_neurons=368; total time=  52.5s
Epoch 1/30
1146/1146 [==============================] - 2s 2ms/step - loss: 2.1809 - accuracy: 0.2598 - val_loss: 2.0087 - val_accuracy: 0.4682
Epoch 2/30
1146/1146 [==============================] - 2s 2ms/step - loss: 1.8473 - accuracy: 0.5828 - val_loss: 1.6735 - val_accuracy: 0.6778
Epoch 3/30
1146/1146 [==============================] - 2s 1ms/step - loss: 1.5237 - accuracy: 0.7175 - val_loss: 1.3541 - val_accuracy: 0.7586
Epoch 4/30
1146/1146 [==============================] - 2s 1ms/step - loss: 1.2401 - accuracy: 0.7727 - val_loss: 1.0979 - val_accuracy: 0.7990
Epoch 5/30
1146/1146 [==============================] - 2s 1ms/step - loss: 1.0261 - accuracy: 0.8021 - val_loss: 0.9156 - val_ac

1719/1719 [==============================] - 2s 934us/step - loss: 0.0370 - accuracy: 0.9909 - val_loss: 0.0687 - val_accuracy: 0.9804
Epoch 24/30
1719/1719 [==============================] - 2s 941us/step - loss: 0.0353 - accuracy: 0.9916 - val_loss: 0.0676 - val_accuracy: 0.9804
Epoch 25/30
1719/1719 [==============================] - 2s 942us/step - loss: 0.0334 - accuracy: 0.9924 - val_loss: 0.0672 - val_accuracy: 0.9798
Epoch 26/30
1719/1719 [==============================] - 2s 932us/step - loss: 0.0317 - accuracy: 0.9930 - val_loss: 0.0675 - val_accuracy: 0.9808
Epoch 27/30
1719/1719 [==============================] - 2s 936us/step - loss: 0.0302 - accuracy: 0.9935 - val_loss: 0.0677 - val_accuracy: 0.9802
Epoch 28/30
1719/1719 [==============================] - 2s 958us/step - loss: 0.0287 - accuracy: 0.9939 - val_loss: 0.0662 - val_accuracy: 0.9808
Epoch 29/30
1719/1719 [==============================] - 2s 989us/step - loss: 0.0274 - accuracy: 0.9942 - val_loss: 0.0660 - val_

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002290FB55520>,
                   param_distributions={'learning_rate': [0.0019317377280177826,
                                                          0.0029044355692588717,
                                                          0.018549775858543845,
                                                          0.00040850794928156835,
                                                          0.0012274695940884513,
                                                          0.0011045516244715366,
                                                          0.0026052914301476606,
                                                          0.000542932243717955,
                                                          0.027950913415857967,
                                                          0.01539745...
                                                          0.00046198415

# Avaliando o melhor modelo

In [13]:
best_model = keras.models.load_model("melhor_modelo_mlp_hiperparameters2.h5")
best_model.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_32 (Flatten)        (None, 784)               0         
                                                                 
 dense_94 (Dense)            (None, 398)               312430    
                                                                 
 dense_95 (Dense)            (None, 398)               158802    
                                                                 
 dense_96 (Dense)            (None, 10)                3990      
                                                                 
Total params: 475,222
Trainable params: 475,222
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Meta: Acurácia:  0.8819 e Kappa:  0.8687777777777779
y_pred = np.argmax(best_model.predict(X_test), axis=-1)

313/313 [==============================] - 0s 773us/step


In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
print('Acurácia: ', accuracy_score(y_test,y_pred))
print('Kappa: ', cohen_kappa_score(y_test,y_pred))
print('Matriz de confusão: \n', confusion_matrix(y_test,y_pred))

Acurácia:  0.981
Kappa:  0.9788805016321037
Matriz de confusão: 
 [[ 969    0    1    1    1    1    2    1    3    1]
 [   0 1125    2    2    0    1    2    1    2    0]
 [   2    0 1013    3    2    0    2    6    4    0]
 [   1    0    3  991    0    4    0    3    5    3]
 [   1    0    3    0  965    0    1    1    1   10]
 [   2    0    0    7    1  869    4    1    5    3]
 [   5    2    2    1    5    3  939    0    1    0]
 [   1    2    8    2    0    0    0 1006    3    6]
 [   5    0    3    6    3    5    2    3  943    4]
 [   2    2    0    4    5    1    0    3    2  990]]


In [16]:
best_model2 = keras.models.load_model("melhor_modelo_mlp_hiperparameters2_funil.h5")
best_model2.summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_63 (Flatten)        (None, 784)               0         
                                                                 
 dense_178 (Dense)           (None, 213)               167205    
                                                                 
 dense_179 (Dense)           (None, 10)                2140      
                                                                 
Total params: 169,345
Trainable params: 169,345
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Meta: Acurácia:  0.8819 e Kappa:  0.8687777777777779
y_pred2 = np.argmax(best_model2.predict(X_test), axis=-1)

313/313 [==============================] - 0s 610us/step


In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
print('Acurácia: ', accuracy_score(y_test,y_pred2))
print('Kappa: ', cohen_kappa_score(y_test,y_pred2))
print('Matriz de confusão: \n', confusion_matrix(y_test,y_pred2))

Acurácia:  0.9795
Kappa:  0.9772126406439988
Matriz de confusão: 
 [[ 972    1    1    0    1    1    1    1    1    1]
 [   0 1125    4    0    0    1    2    1    2    0]
 [   4    3 1003    5    3    0    1    8    5    0]
 [   0    0    2  998    0    1    0    2    3    4]
 [   1    0    3    1  963    0    2    2    0   10]
 [   2    0    0   12    2  866    4    1    3    2]
 [   5    3    2    1    4    3  939    1    0    0]
 [   2    3    5    3    0    0    0 1008    2    5]
 [   6    1    2    8    5    2    2    4  942    2]
 [   4    4    0    5    8    1    0    6    2  979]]
